In [1]:
import numpy as np
import csv
import json
import os 
import datetime
import matplotlib.pyplot as plt
import seaborn as sb
import shutil
from importlib import reload
from IPython.display import Image, display, HTML

import pprint 
pp = pprint.PrettyPrinter(indent=4)

In [2]:
import config
import utils

import raw_log_types
import results
import tasks
import task_results
import main

reload(main)

<module 'main' from 'c:\\Users\\frank\\source\\repos\\vbs2021-analysis\\main.py'>

In [3]:
vbs = main.VbsVis()
main.VbsVis.flush_cache()

%%% PARSING TASKS %%%
%%% Parsed 38 tasks. %%%
%%% DONE! %%%
./__cache__/
Error while deleting directory


# Results

In [4]:
#data.print_tasks()

In [5]:
vbs.parse_team("SOMHunter", 
                  ["sh-patrik", "sh-vit"], 
                  verbose=True, 
                  validate_fix=False,
                  generate_DRES=False
                 )


TEAM: SOMHunter

%%% PARSING RESULTS %%%
---
	 +++ sh-patrik +++ 
	DATA: ./data/sh-patrik 

['road shadow']
['road shadow']
['road curb']
['road curb']
['flag']
['flag']
['flag']
['flag']
['crowd']
['graffiti']
['graffiti']
['graffiti']
['graffiti purple']
['graffiti purple']
['spray blue jacket']
['motorbike dirt road']
['dirt road']
['buggy dirt road']
['buggy dirt road']
['buggy dirt road']
['buggy dirt road']
['buggy dirt road']
['buggy dirt road']
['buggy dirt road']
['beer']
['beer']
['beer']
['beer drinking']
['window hand']
['window snow mountain']
['window snow mountain']
['snow mountain shadow']
['snow mountain hut']
['window snow']
['window snow window']
['hand snow']
['hand snow window']
['hand snow window']
['snow hand']
['snow mountain', 'snow mountain']
TEMP
['snow mountain', 'snow mountain']
TEMP
['young eating']
['young eating']
['young eating']
['trees evening eating young']
['trees evening eating young eating']
['headphones eating']
['headphones eating eating']
['hea

['man body shoes shirt naked bathroom', 'woman bride']
TEMP
['man getting dressed', 'woman bride']
TEMP
['ski']
['skate', 'ticket']
TEMP
['skate', 'ticket ticket']
TEMP
['skateboard man', 'ticket ticket']
TEMP
['skateboard man', 'ticket ticket a8']
TEMP
['skateboard man', 'ticket ticket airport']
TEMP
['man holding skateboard elevator', 'ticket ticket airport']
TEMP
['man holding skateboard elevator elevator', 'ticket ticket airport']
TEMP
['man holding skateboard elevator elevator stairs', 'ticket ticket airport']
TEMP
['man holding skateboard elevator elevator stairs', 'ticket ticket airport']
TEMP
['man holding skateboard elevator elevator stairs', 'ticket ticket airport']
TEMP
['man holding skateboard elevator elevator stairs', 'ticket ticket airport']
TEMP
['man holding skateboard elevator elevator stairs', 'ticket ticket airport']
TEMP
['man holding skateboard elevator elevator stairs', 'skateboard']
TEMP
['man holding skateboard elevator elevator stairs', 'skateboard']
TEMP
['ma

In [6]:
vbs.print_task_results(["SOMHunter"])
vbs.print_task_results()

***############################***
***###***TASK RESULT LOGS***###***
***############################***
--- SOMHunter ---
	--- sh-patrik ---
		--- 01_v21-1 ---

>>> 14.07 <<<
- ['TQ'] -
- (70, 10000000, 19000) -
------------------------

>>> 34.28 <<<
- ['LK', 'TQ'] -
- (39, 10000000, 18940) -
------------------------

>>> 66.44 <<<
- ['TQ'] -
- (47, 102, 18548) -
------------------------

>>> 86.84 <<<
- ['LK', 'TQ'] -
- (9, 23, 18387) -
------------------------

>>> 207.30 <<<
- ['CTQ'] -
- (404, 1363, 18952) -
------------------------

>>> 225.21 <<<
- ['CTQ'] -
- (1913, 10000000, 18874) -
------------------------

>>> 239.30 <<<
- ['CTQ'] -
- (392, 10000000, 18842) -
------------------------

>>> 247.18 <<<
- ['NN'] -
- (70, 10000000, 10000) -
------------------------

>>> 247.26 <<<
- ['CTQ'] -
- (392, 10000000, 18842) -
------------------------

>>> 265.30 <<<
- ['CTQ', 'LK'] -
- (80, 10000000, 18733) -
------------------------

		--------------------

		--- 02_a21-5 ---

>>> 14

In [7]:
vbs.print_tool_features(["SOMHunter"])
vbs.print_tool_features()


***#####################################***
***###*** TOOL FEATURES & STATES  ***###***
***#####################################***
--- SOMHunter ---
	['TQ'] -> temporal-text-query
	['TQR', 'LK', 'FDP'] -> filter-dataset-part, like, text-query-relocation


KeyError: 'TTQ'

In [ ]:
# vbs.parse_team("CollageHunter", 
#                   ["collage-jakub", "collage-premek"], 
#                   verbose=True, 
#                   validate_fix=False, 
#                   generate_DRES=False
#                  )

In [ ]:
# vbs.parse_team("LegacySOMHunter", 
#                   ["legacy-tereza", "legacy-franta-tomas"], 
#                   verbose=True, 
#                   validate_fix=False, 
#                   generate_DRES=False
#                  )

In [ ]:
x = "|results| [wheel ]  [blurred ]  >> cap face;fromVideoLimit=3;fromShotLimit=1;datasetPartsFilter=true,true,;" 

In [ ]:

    
extract_text_query(x)

In [ ]:
vbs.print_task_results()

In [ ]:
def plot_result_charts(self, team_ID, member_IDs, novice_mode = 'a'):      
        
    COLORS = {
        1: "#9833FF",
        2: "#FF8A38"
    }
    
    d_marker_random = {1:{"marker": r"$R$", "markerfacecolor": COLORS[1], "markeredgecolor": COLORS[1], "markersize": 5},
                       2:{"marker": r"$R$", "markerfacecolor": COLORS[2], "markeredgecolor": COLORS[2], "markersize": 5}}
    
    d_marker_topn = {1:{"marker": r"$T$", "markerfacecolor": COLORS[1], "markeredgecolor": COLORS[1], "markersize": 5},
                     2:{"marker": r"$T$", "markerfacecolor": COLORS[2], "markeredgecolor": COLORS[2], "markersize": 5}}
                         
    d_marker_topknn = {1:{"marker": r"$N$", "markerfacecolor": COLORS[1], "markeredgecolor": COLORS[1], "markersize": 5},
                       2:{"marker": r"$N$", "markerfacecolor": COLORS[2], "markeredgecolor": COLORS[2], "markersize": 5}}
    
    d_marker_som = {1:{"marker": r"$S$", "markerfacecolor": "#000000", "markeredgecolor": "#000000", "markersize": 5},
                    2:{"marker": r"$S$", "markerfacecolor": "#000000", "markeredgecolor": "#000000", "markersize": 5}}
    
    d_marker_detail = {1:{"marker": r"$D$", "markerfacecolor": COLORS[1], "markeredgecolor": COLORS[1], "markersize": 5},
                       2:{"marker": r"$D$", "markerfacecolor": COLORS[2], "markeredgecolor": COLORS[2], "markersize": 5}}
    
    d_marker_replay = {1:{"marker": r"$P$", "markerfacecolor": COLORS[1], "markeredgecolor": COLORS[1], "markersize": 5},
                       2:{"marker": r"$P$", "markerfacecolor": COLORS[2], "markeredgecolor": COLORS[2], "markersize": 5}}
    
    
    empty_marker = {"marker": ",", "markerfacecolor": "#000000", "markeredgecolor": "#000000", "markersize": 12}
    
    text_marker = {"marker": r"$t$", "markerfacecolor": "#000000", "markeredgecolor": "#000000", "markersize": 12}
    text_marker_red = {"marker": r"$t$", "markerfacecolor": "#ff0000", "markeredgecolor": "#ff0000", "markersize": 12}
    
    temp_marker = {"marker": r"$T$", "markerfacecolor": "#000000", "markeredgecolor": "#000000", "markersize": 12}
    temp_marker_red = {"marker": r"$T$", "markerfacecolor": "#ff0000", "markeredgecolor": "#ff0000", "markersize": 12}
    
    asr_marker = {"marker": r"$A$", "markerfacecolor": "#000000", "markeredgecolor": "#000000", "markersize": 15}
    asr_marker_red = {"marker": r"$A$", "markerfacecolor": "#ff0000", "markeredgecolor": "#ff0000", "markersize": 15}
    
    knn_marker_bkg = {"marker": "D", "markerfacecolor": "#FFFFFF", "markeredgecolor": "#226BA6", "markersize": 18}
    knn_marker_bkg_red = {"marker": "D", "markerfacecolor": "#FFFFFF", "markeredgecolor": "#ff0000", "markersize": 18}
    
    empty_marker_bkg = {"marker": None, "markerfacecolor": "#000000", "markeredgecolor": "#000000", "markersize": 0}
    
    bayes_marker_bkg = {"marker": "s", "markerfacecolor": "#FFFFFF", "markeredgecolor": "#226BA6", "markersize": 18}
    bayes_marker_bkg_red = {"marker": "s", "markerfacecolor": "#FFFFFF", "markeredgecolor": "#ff0000", "markersize": 18}
    
    color_marker_bkg = {"marker": "o", "markerfacecolor": "#FFFFFF", "markeredgecolor": "#226BA6", "markersize": 16}
    color_marker_bkg_red = {"marker": "o", "markerfacecolor": "#FFFFFF", "markeredgecolor": "#ff0000", "markersize": 16}
    
    localized_marker_bkg = {"marker": "v", "markerfacecolor": "#FFFFFF", "markeredgecolor": "#226BA6", "markersize": 16}
    localized_marker_bkg_red = {"marker": "v", "markerfacecolor": "#FFFFFF", "markeredgecolor": "#ff0000", "markersize": 16}
    
    
    
        
    # a/n/e
        
    for task_name, task in self.data.items():
        
        is_novice = task["is_novice"]
        
        if (novice_mode != "a"):
            if (novice_mode == "n"):
                if (not is_novice): continue
            elif (novice_mode == "e"):
                if (is_novice): continue
                    
        times = {}
        vals_videos = {}
        vals_frames = {}
        
        result_num_limit = {}
        
        keys_corr_submits = {}
        keys_incorr_submits = {}
        
        d_keys = {}
        d_vals = {}
        
        # Flags
        flags = {}
        
        for member_ID in member_IDs:
            
            # Input data
            times[member_ID] = task["results"][team_ID][member_ID][0]
            vals_videos[member_ID] = task["results"][team_ID][member_ID][1]
            vals_frames[member_ID] = task["results"][team_ID][member_ID][2]
            flags[member_ID] = task["results"][team_ID][member_ID][3]
            result_num_limit[member_ID] = task["results"][team_ID][member_ID][4]
            
            keys_corr_submits[member_ID] = task["submits"][team_ID][member_ID]["correct"]
            keys_incorr_submits[member_ID] = task["submits"][team_ID][member_ID]["incorrect"]
            
           

            
        
        # Canvas size
        fig, ax = plt.subplots(figsize=(16,4))

        # Title
        ax.set_title(task["paper_name"] + " (" + task_name + ")", y=1.0)

        #ax.yaxis.set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(True)
        ax.spines['top'].set_visible(False)
        ax.xaxis.set_ticks_position('bottom')


        # Set y to log scale
        plt.yscale("log")

        # Set y axis range
        plt.ylim(1, 40000)
        ax.set_xlim(xmin=0, xmax=500)


        for member_ID in member_IDs:
            
            
            
            
            ##########################
            # Draw video & frame pos
            ##########################
            #
            # Video positions
            #
            plt.step(times[member_ID], vals_videos[member_ID], where='post', label='post', color=COLORS[member_ID] , linewidth="2")
            
            #
            # Frame positions
            #
            plt.step(times[member_ID], vals_frames[member_ID], where='post', label='post', color=COLORS[member_ID], linestyle=':', linewidth="4")
            
            #
            # Limit of send results
            #
            plt.step(times[member_ID], result_num_limit[member_ID], where='post', label='post', color=COLORS[member_ID], linestyle='dashdot', linewidth="1")
            
            ##########################
            # Draw display switches
            ##########################

            if (team_ID in task["displays"] and member_ID in task["displays"][team_ID]):
                displ_types = task["displays"][team_ID][member_ID][1]

                for state in displ_types:
                    #plt.axvspan(i, i+.5, facecolor='b', alpha=0.5)

                    if (state.display == "R"):
                        plt.plot([state.time], [2000 + member_ID * 1000], **d_marker_random[member_ID])
                    elif state.display == "T":
                        plt.plot([state.time], [2000 + member_ID * 1000], **d_marker_topn[member_ID])
                    elif state.display == "N":
                        plt.plot([state.time], [2000 + member_ID * 1000], **d_marker_topknn[member_ID])
                    elif state.display == "S":
                        plt.plot([state.time], [2000 + member_ID * 1000], **d_marker_som[member_ID])
                    elif state.display == "D":
                        plt.plot([state.time], [2000 + member_ID * 1000], **d_marker_detail[member_ID])
                    elif state.display == "P":
                        plt.plot([state.time], [2000 + member_ID * 1000], **d_marker_replay[member_ID])

            
            ##########################
            # Draw event markers
            ##########################

            iii = 0
            for f in flags[member_ID]:
                
                keys = f.keys()
                rich = f["is_knn"] or f["is_localized"] or f["is_color"] or f["is_bayes"]or f["is_text"]
                
                
                if not rich:
                    cust_marker = {"marker": r"$?$", "markerfacecolor": COLORS[member_ID], "markeredgecolor": COLORS[member_ID], "markersize": 12}
                    plt.plot([times[member_ID][iii]], [vals_videos[member_ID][iii]], **cust_marker)
                
                #
                # Is KNN
                #
                if (f["is_asr"]):
                    plt.plot([times[member_ID][iii]], [vals_videos[member_ID][iii]], **asr_marker)
                    
                        
                 #
                # Is KNN
                #
                if (f["is_knn"]):
                    if (f["is_knn"]==1):
                        plt.plot([times[member_ID][iii]], [vals_videos[member_ID][iii]], **knn_marker_bkg)
                    else:
                        plt.plot([times[member_ID][iii]], [vals_videos[member_ID][iii]], **knn_marker_bkg_red)
                        
                #
                # Is localized object
                #
                if (f["is_localized"]):
                    if (f["is_localized"]==1):
                        plt.plot([times[member_ID][iii]], [vals_videos[member_ID][iii]], **localized_marker_bkg)
                    else:
                        plt.plot([times[member_ID][iii]], [vals_videos[member_ID][iii]], **localized_marker_bkg_red)
                #
                # Is color sketch
                #
                if (f["is_color"]):
                    if (f["is_color"]==1):
                        plt.plot([times[member_ID][iii]], [vals_videos[member_ID][iii]], **color_marker_bkg)
                    else:
                        plt.plot([times[member_ID][iii]], [vals_videos[member_ID][iii]], **color_marker_bkg_red)
                    
                
                    
               
                    
                #
                # Is Bayes rerank
                #
                if (f["is_bayes"]):
                    if (f["is_bayes"]==1):
                        plt.plot([times[member_ID][iii]], [vals_videos[member_ID][iii]], **bayes_marker_bkg)
                    else:
                        plt.plot([times[member_ID][iii]], [vals_videos[member_ID][iii]], **bayes_marker_bkg_red)
                    
                #
                # (Temporal) Text query
                #
                if (f["is_text"]):
                    
                    if ("text_type" in f and f["text_type"] != None):
                        label = ""
                        
                        if f["text_type"]["concept"]:
                            label += "C,"
                            
                        if f["text_type"]["caption"]:
                            label += "P,"
                            
                        if f["text_type"]["ocr"]:
                            label += "O,"
                        
                        if (label != ""):
                            cust_marker = {"marker": r"$t$", "markerfacecolor": "#000000", "markeredgecolor": "#000000", "markersize": 12, "label":label}                        
                            plt.plot([times[member_ID][iii]], [vals_videos[member_ID][iii]], **cust_marker)
                            plt.text(times[member_ID][iii] + 2, vals_videos[member_ID][iii], label, fontsize=10)
                            
                        
                        
                        
                    else:
                        if (f["is_text"] == 1):
                            if (f["is_temporal"]):
                                plt.plot([times[member_ID][iii]], [vals_videos[member_ID][iii]], **temp_marker)
                            else:
                                plt.plot([times[member_ID][iii]], [vals_videos[member_ID][iii]], **text_marker)
                        else:
                            if (f["is_temporal"]):
                                plt.plot([times[member_ID][iii]], [vals_videos[member_ID][iii]], **temp_marker_red)
                            else:
                                plt.plot([times[member_ID][iii]], [vals_videos[member_ID][iii]], **text_marker_red)
                


                    
                iii +=1
            
            ##########################
            # Draw end of the task
            ##########################
            plt.axvline(x=task["length"], color='r', linestyle=':', linewidth="10")

            ##########################
            # Draw submits in
            ##########################
            for s in keys_corr_submits[member_ID]:
                plt.axvline(x=s, color='g', linewidth="5")

            #print(keys_incorr_submits)
            for s in keys_incorr_submits[member_ID]:
                plt.axvline(x=s, color='r')

    
        #ax.get_yaxis().set_ticklabels([])
        plt.show()
        
    plt.savefig(CONFIG.BASE_DIR + "timelines.png")